<a href="https://colab.research.google.com/github/GiovanniSorice/Hate_Speech_Detection/blob/main/notebooks/scraper_tweet_evaluate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [92]:
!pip install tokenizer
!pip install ekphrasis
!pip install wordninja
!pip install emoji
!pip install spacy_udpipe
!pip install language_tool_python
!pip install compound-word-splitter
!pip install pyenchant
!sudo apt-get install enchant
!sudo apt-get install hunspell-it
!pip install transformers

Reading package lists... Done
Building dependency tree       
Reading state information... Done
enchant is already the newest version (1.6.0-11.1).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
hunspell-it is already the newest version (1:6.0.3-3).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.
     |████████████████████████████████| 2.0MB 5.0MB/s 
     |████████████████████████████████| 3.2MB 13.4MB/s 
     |████████████████████████████████| 890kB 36.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=16fa9865a1ab081cecc7dea4ed6299faa68f69aaec0ea5f80bc233507199117b
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [98]:
import pandas as pd
import csv
import nltk
import re
import matplotlib.pyplot as plt
from nltk.tokenize import TweetTokenizer
from tokenizer import *
from nltk.corpus import stopwords

from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons

import wordninja

from nltk.stem import SnowballStemmer

import ast

import emoji
import unicodedata

import gzip

import spacy_udpipe
import language_tool_python

import splitter
import enchant
from itertools import groupby
import string

import sys
import os

#KimCNN
import tensorflow as tf
import numpy as np

# ALBERTo
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons
import torch
from torch.utils.data import TensorDataset,DataLoader, RandomSampler, SequentialSampler

pd.set_option("display.max_colwidth", None)

In [3]:
with_drive = True

if with_drive:
  from google.colab import drive
  # This will prompt for authorization.
  drive.mount('/content/drive')

Mounted at /content/drive


In [44]:
def preprocessing_KimCNN_BiLSTM(dataset_path, bad_words_path):
  def comment_length(text):
    return len(text)

  def caps_lock_words(text):
    words = text.split()
    count_caps_lock = 0
    number_of_words = len(words)
    
    for word in words:
        if word.isupper() == True:
            count_caps_lock = count_caps_lock + 1
            
    return ((count_caps_lock*100)//number_of_words)

  def clean_tag(text):
    return re.sub(
        r'@user', ' ', text)
    
  def replace_e_a(text):
    text = re.sub(r'&', 'e', text)
    return re.sub(r'@', 'a', text)

  def clean_disguised_bad_words(text):
    text = " " + text + " "
    text = re.sub(r' c[.x*@%#$^]+i ', ' coglioni ', text)
    text = re.sub(r' c[.x*@%#$^]+e ', ' coglione ', text)
    text = re.sub(r' c[.x*@%#$^]+o ', ' cazzo ', text) 
    text = re.sub(r' ca[.x*@%#$^]+ro ', ' cazzaro ', text) 
    text = re.sub(r' c[.x*@%#$^]+i ', ' cazzi ', text) 
    text = re.sub(r' m[.x*@%#$^]+a ', ' merda ', text) 
    text = re.sub(r' m[.x*@%#$^]+e ', ' merde ', text) 
    text = re.sub(r' c[.x*@%#$^]+ulo ', ' culo ', text) 
    text = re.sub(r' p[.x*@%#$^]+a ', ' puttana ', text)
    text = re.sub(r' p[.x*@%#$^]+e ', ' puttane ', text)
    text = re.sub(r' t[.x*@%#$^]+a ', ' troia ', text)
    text = re.sub(r' t[.x*@%#$^]+e ', ' troie ', text)
    text = re.sub(r' s[.x*@%#$^]+o ', ' stronzo ', text)
    text = re.sub(r' s[.x*@%#$^]+i ', ' stronzi ', text)
    return text

  def find_hashtags(text):
    result = re.findall(r'#\S+', text)
    if result:
        return result
    else:
        return None

  def split_hashtags(text):
    
    text = ' ' + text + ' '
    result = re.findall(r'#\S+', text)
    
    for word in result:
        new_word = " ".join(splitter.split(word[1:].lower(), 'it_IT'))
        if len(new_word)==0:
            new_word =  word[1:]
                  
        text = text.replace(word, new_word)
        
    return text

  def clean_URL(text):
    return re.sub(r'URL', ' ', text)

  def esclamations_and_questions(text):
    return text.count('!') + text.count('?')

  def esclamations_and_questions(text):
      return text.count(',') + text.count('.')

  def strip_punctuation(text):
    regex = re.compile('[%s]' % re.escape(string.punctuation))
    return regex.sub(' ', text)

  correct_words_vowels = ['sciiti,',
  'welcomerefugees',
  'livetweet',
  'desiree',
  'canaan',
  'tweet.',
  'weekend',
  'romafeyenoord',
  'ebree,',
  'greencard',
  'creerà',
  'cooperative.',
  'moschee,',
  'cooperanti',
  'streetart',
  'khalidmasood',
  'tweet',
  'woolfe',
  'cooperazione',
  'coop',
  'seehofer',
  'speech',
  'coffee',
  'scooter',
  'street',
  'veemenza',
  'moschee.',
  'maalox.',
  'book',
  'tweet',
  'facebook:',
  'sociial,',
  'coop,',
  'canaaniti.',
  'europee,',
  'cooperative',
  'google',
  'creeranno',
  'mediterranee',
  'cooperazione',
  'cooperativa',
  '“boom”',
  'refugees',
  'moonlight',
  'imaam',
  'shooting',
  'sciiti',
  'sunniti',
  'book',
  'atee.',
  'looking',
  'week',
  'ayaan',
  'temporanee.',
  'idee.',
  'sibiliini',
  'food',
  'refugees',
  'retweeted',
  'boom',
  'keep',
  'vodoo',
  'hooligans',
  'ebree',
  'refugees',
  'speed',
  'bloomberg',
  'riina',
  'hatespeech',
  'google',
  'masood',
  'linee.',
  'boom']
  def delete_duplicate_vowels_and_redundant_consonant (text):
      parole = text.split()
      stringa = ""
      for a in parole:
          parola = a
          a = [list(g) for k, g in groupby(a)]    
          vocali = ['a','e','i','o','u','y']
          
          for idx,val in enumerate(a):
              if idx == 0:
                  stringa += a[idx][0] 
              elif idx == len(a)-1:
                  stringa += a[idx][0]
              elif a[idx][0] in vocali and (parola.lower() not in correct_words_vowels):
                  stringa += a[idx][0]
              elif len(a[idx]) == 1:
                  stringa += a[idx][0]
              elif len (a[idx]) >= 2:
                  stringa += a[idx][0]
                  stringa += a[idx][1]
          stringa =  stringa + " "
          
      return(stringa)

  def translate_emoticon(text):
    text_result = emoji.demojize(text, language='it')
    text_result=re.sub(r':', ' ', text_result)
    return text_result

  emoticons_text = {
    '<kiss>': 'bacio',
    '<happy>': 'felice',
    '<laugh>': 'risata',
    '<sad>': 'triste',
    '<surprise>': 'sorpreso',
    '<wink>': 'occhiolino',
    '<tong>': 'faccia con lingua',
    '<annoyed>': 'annoiato',
    '<seallips>': 'labbra sigillate',
    '<angel>': 'angelo',
    '<devil>': 'diavolo',
    '<highfive>' : 'batti il cinque',
    '<heart>': 'cuore',
    '<user>' : 'persona',
  }

  def clean_emoticon_text(text):
    text_words = text.split()
    new_words  = [emoticons_text.get(ele, ele) for ele in text_words]
    return ' '.join(new_words)

  abbr_word = {'cmq':'comunque', 'gov':'governatori', 'fb':'facebook', 'tw':'twitter', 'juve':'juventus', 'ing':'ingegnere', 
             'sx':'sinistra', 'qdo':'quando', 'rep':'repubblica', 'grz':'grazie', 'ita':'italia', 'mln':'milioni', 
             'mld':'miliardi', 'pke':'perche', 'anke':'anche', 'cm':'come', 'dlla':'della', 'dlle':'delle', 'qst':'questa',
             'ke':'che', 'nn':'non', 'sn':'sono', 'cn':'con', 'xk':'perche', 'xke':'perche', 'art':'articolo',
             'tv':'televisore', '€':'euro', 'xché':'perché', 'xké':'perché', 'pkè':'perché'}
             
  def replace_abbreviation(text):
    text_words = text.split()
    new_text = ""
    for token in text_words:
      new_text  += abbr_word.get(token, token) +" "
    
    return new_text.strip()

  laughs = ['ah', 'eh', 'he' 'ih', 'hi'] #non elimina ahahahah, ma solo ah
  vowels = ['a', 'e', 'i', 'o', 'u']

  def clean_laughs(text):
      #s = "ahahahah ho fame io, eh eh" -> " ho fame io,"
      text_words = text.split()
      new_words  = [word for word in text_words if word not in laughs]
      
      new_text = ' '.join(new_words)
      
      for i in new_words:
          for k in vowels:
              if ('h' in i) and (len(i) >= 4):
                  if (len(i) - 2) <= (i.count(k) + i.count('h')):
                      new_text = new_text.replace(i, '')
      
      return new_text

  def tokenization(text):
    tknzr=SocialTokenizer(lowercase=False)
    return tknzr.tokenize(text)

  def number_bad_words(tokens):
      n_bad_words = 0

      for word in tokens:
          if word.lower() in bad_words_set:
              n_bad_words = n_bad_words + 1
          
      return n_bad_words

  def percentage_bad_words(tokens):
    n_words = 0
    n_bad_words = 0
    
    for word in tokens:
        if word != '<' and word != '>':
            n_words = n_words + 1
    
    for word in tokens:
        if word.lower() in bad_words_set:
            n_bad_words = n_bad_words + 1
        
    return ((n_bad_words*100)//n_words)


  csv_file = open(dataset_path)

  dataset_raw = pd.read_csv(csv_file,sep=',', index_col=0)

  dataset_raw.rename(columns={"Text": "text"}, inplace=True) 
   #Bad Words
  f2 = open(bad_words_path, 'r', encoding='utf8')

  bad_words_set = [] #list of lowercase words

  for x in f2:
      y = x.rstrip()
      y = y.lower()
      if y != '':
          bad_words_set.append(y)

  dataset_raw['text_length'] = dataset_raw['text'].apply(comment_length)
  dataset_raw['#C-L words'] = dataset_raw['text'].apply(caps_lock_words)
  dataset_raw['text'] = dataset_raw['text'].apply(clean_tag)
  dataset_raw['text'] = dataset_raw['text'].apply(replace_e_a)
  dataset_raw['text'] = dataset_raw['text'].apply(clean_disguised_bad_words)
  dataset_raw['text'] = dataset_raw['text'].apply(clean_disguised_bad_words)
  dataset_raw['hashtags'] = dataset_raw['text'].apply(find_hashtags)
  dataset_raw['text'] = dataset_raw['text'].apply(split_hashtags)
  dataset_raw['text'] = dataset_raw['text'].apply(clean_URL)
  dataset_raw['#?!'] = dataset_raw['text'].apply(esclamations_and_questions)
  dataset_raw['#.,'] = dataset_raw['text'].apply(esclamations_and_questions)
  dataset_raw['text'] = dataset_raw['text'].apply(strip_punctuation)
  dataset_raw['text'] = dataset_raw['text'].apply(delete_duplicate_vowels_and_redundant_consonant)
  dataset_raw['text'] = dataset_raw['text'].apply(translate_emoticon)
  dataset_raw['text'] = dataset_raw['text'].apply(clean_emoticon_text)
  dataset_raw['text'] = dataset_raw['text'].apply(replace_abbreviation)
  dataset_raw['text'] = dataset_raw['text'].apply(clean_laughs)
  dataset_raw['tokens'] = dataset_raw['text'].apply(tokenization)
  dataset_raw['#bad_words'] = dataset_raw['tokens'].apply(number_bad_words)
  dataset_raw['%bad_words'] = dataset_raw['tokens'].apply(percentage_bad_words)

  return dataset_raw

In [45]:
parole_non_ric = set()
def sentence_to_emb2(sentence, w2v, truncate = None, padding = False):
  global parole_non_ric
  pad_token = [0]*128
  s_emb = [ w2v[word.lower()] for word in sentence if word.lower() in w2v.vocab]
  parole_non_ric.update(set([ word.lower() for word in sentence if word.lower() not in w2v.vocab]))
  if truncate is not None:
    s_emb = s_emb[:truncate] #truncate
  if padding:
    s_emb += [pad_token] * (truncate - len(s_emb))
  return np.array(s_emb)

def get_data_to_emb2(data, w2v, truncate = None, padding = False):
  X = [sentence_to_emb2(sentence, w2v, truncate, padding) for sentence in data]
  return np.array(X)

In [89]:
def preprocessing_ALBERTo(dataset_path):
  text_processor = TextPreProcessor(
    # terms that will be normalized
    normalize=['url', 'email', 'user', 'percent', 'money', 'phone', 'time', 'date', 'number'],
    # terms that will be annotated
    annotate={"hashtag"},
    fix_html=True,  # fix HTML tokens

    unpack_hashtags=True,  # perform word segmentation on hashtags

    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=True).tokenize,
    dicts=[emoticons]
    )

  class AlBERTo_Preprocessing(object):
      def __init__(self, do_lower_case=True, **kwargs):
          self.do_lower_case = do_lower_case

      def preprocess(self, text):
          if self.do_lower_case:
              text = text.lower()
          text = str(" ".join(text_processor.pre_process_doc(text)))
          text = re.sub(r'[^a-zA-ZÀ-ú</>!?♥♡\s\U00010000-\U0010ffff]', ' ', text)
          text = re.sub(r'\s+', ' ', text)
          text = re.sub(r'(\w)\1{2,}', r'\1\1', text)
          text = re.sub(r'^\s', '', text)
          text = re.sub(r'\s$', '', text)
          return text

  AlBERTo_Preprocess = AlBERTo_Preprocessing(do_lower_case=True)
  def preprocess(text):
      return AlBERTo_Preprocess.preprocess(text)

  tokenizer = BertTokenizer.from_pretrained('m-polignano-uniba/bert_uncased_L-12_H-768_A-12_italian_alb3rt0')

  csv_file = open(dataset_path)

  dataset_raw = pd.read_csv(csv_file,sep=',', index_col=0)

  dataset_raw.rename(columns={"Text": "text"}, inplace=True) 


  dataset_raw['text_preprocessed'] = dataset_raw['text'].apply(preprocess)
  encoding_dataset = tokenizer(list(dataset_raw["text_preprocessed"].values), padding=True, truncation=True, max_length=64)
  input_ids_dataset = torch.tensor(encoding_dataset['input_ids'])
  attention_mask_dataset = torch.tensor(encoding_dataset['attention_mask'])

  test_tweets_data = TensorDataset(input_ids_dataset, attention_mask_dataset)
  test_tweets_sampler = SequentialSampler(test_tweets_data)
  test_tweets_dataloader = DataLoader(test_tweets_data, sampler=test_tweets_sampler, batch_size=len(test_tweets_data))

  return test_tweets_dataloader

In [46]:
def predict_with_ensemble_tf(models, test_input):
  # make predictions
  results = []
  y_predict = [np.squeeze(np.where(model.predict(test_input) > 0.5, 1,0).reshape(1,-1)) for model in models]
  # sum across ensemble members
  y_predict = np.array(y_predict)

  for i in range(y_predict.shape[1]):
    counts = np.bincount(y_predict[:,i])
    results.append(np.argmax(counts))
  # argmax across classes
  return results

In [101]:
def evaluate_for_kfold_torch(model, val_dataloader):
    """After the completion of each training epoch, measure the model's performance
    on our validation set.
    """
    # Put the model into the evaluation mode. The dropout layers are disabled during
    # the test time.
    model.eval()
    preds = []
    # Tracking variables
    val_accuracy = []
    val_loss = []
    f1_value = []
    # For each batch in our validation set...
    for batch in val_dataloader:
        # Load batch to GPU
        b_input_ids, b_attn_mask = tuple(t.to(device) for t in batch)

        # Compute logits
        with torch.no_grad():
            logits = model(b_input_ids, b_attn_mask)
        # Get the predictions
        preds+=(torch.argmax(logits, dim=1).flatten().tolist())
    
    return preds

In [102]:
def predict_with_ensemble_torch(models, test_dataloader):
  # make predictions
  results = []
  y_predict = []
  for model in models:
    model.to(device)
    y_predict.append(evaluate_for_kfold_torch(model, test_dataloader))
    model.to("cpu")

  #y_predict = [np.squeeze(evaluate_for_kfold(model, test_dataloader)) for model in models]
  # sum across ensemble members
  y_predict = np.array(y_predict)

  for i in range(y_predict.shape[1]):
    counts = np.bincount(y_predict[:,i])
    results.append(np.argmax(counts))
  # argmax across classes
  return results

In [48]:
def evaluate_with_KimCNN(dataset_path, bad_words_path,models_path_list):
  print("Evaluate_with_KimCNN")
  # Preprocessing
  dataset = preprocessing_KimCNN_BiLSTM(dataset_path, bad_words_path)
  # Split dataset in embeddings and other feature
  X_dataset = get_data_to_emb2(dataset["tokens"], w2v, max_length , True)
  dataset_other = dataset.drop(['text','tokens', 'hashtags'], axis=1)

  inputs   = {"text": X_dataset, "other": dataset_other.values}

  # Load models
  models = []
  for path in models_path_list:
    models.append(tf.keras.models.load_model(path))

  return predict_with_ensemble_tf(models, inputs)

In [109]:
def evaluate_with_BiLSTM(dataset_path, bad_words_path, models_path_list):
  print("evaluate_with_BiLSTM")
  # Preprocessing
  dataset = preprocessing_KimCNN_BiLSTM(dataset_path, bad_words_path)
  # Split dataset in embeddings and other feature
  X_dataset = get_data_to_emb2(dataset["tokens"], w2v, max_length , True)
  dataset_other = dataset.drop(['text','tokens', 'hashtags'], axis=1)

  inputs   = {"text": X_dataset, "other": dataset_other.values}

  # Load models
  models = []
  for path in models_path_list:
    models.append(tf.keras.models.load_model(path))

  return predict_with_ensemble_tf(models, inputs)


In [110]:
def evaluate_with_ALBERTo(dataset_path, models_path_list):
  print("evaluate_with_ALBERTo")
  dataset = preprocessing_ALBERTo(dataset_path)

  # Load models
  models = []
  for path in models_path_list:
    models.append(model = torch.load(path))

  return predict_with_ensemble_torch(models, inputs)


In [52]:
from gensim.models import KeyedVectors
from gensim.test.utils import datapath
w2v_path = "/content/drive/My Drive/HLT/w2v/twitter128.bin"
w2v = KeyedVectors.load_word2vec_format(datapath(w2v_path), binary=True)
max_length = 64

In [112]:
dataset_path = '/content/drive/My Drive/HLT/scraper/tweets_perchetendenza.csv'
bad_words_path = '/content/drive/My Drive/HLT/preprocessing/bad_words.csv'
base_model_path_KimCNN = '/content/drive/MyDrive/HLT/clean_dataset_training/model_output/kimCNN/512_0.0_nadam_0.0002/' 
base_model_path_BiLSTM = '/content/drive/MyDrive/HLT/clean_dataset_training/model_output/biLSTM/64_0.2_nadam_0.0/' 
base_model_path_ALBERTo = '/content/drive/MyDrive/HLT/alberto/dr_0.5_dr_0.7/' 
model_path_list_KimCNN = []
model_path_list_BiLSTM = []
model_path_list_ALBERTo = []
for i in range(5):
  model_path_list_KimCNN.append(base_model_path_KimCNN + "model_{0}.h5".format(i))
for i in range(5):
  model_path_list_BiLSTM.append(base_model_path_BiLSTM + "model_{0}.h5".format(i))
for i in range(4):
  model_path_list_ALBERTo.append(base_model_path_ALBERTo + "model_{0}".format(i))

In [63]:
evaluate_with_KimCNN(dataset_path, bad_words_path, model_path_list_KimCNN)

Evaluate_with_KimCNN


11 out of the last 11 calls to <function Model.make_predict_function.<locals>.predict_function at 0x7fb829d04a70> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
11 out of the last 11 calls to <function Model.make_predict_function.<locals>.predict_function at 0x7fb829359830> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing 

[0, 0, 0, 0, 0, 0, 0, 0]

In [83]:
evaluate_with_BiLSTM(dataset_path, bad_words_path, model_path_list_BiLSTM)

evaluate_with_BiLSTM
/content/drive/MyDrive/HLT/clean_dataset_training/model_output/biLSTM/64_0.2_nadam_0.0/model_0.h5
/content/drive/MyDrive/HLT/clean_dataset_training/model_output/biLSTM/64_0.2_nadam_0.0/model_1.h5
/content/drive/MyDrive/HLT/clean_dataset_training/model_output/biLSTM/64_0.2_nadam_0.0/model_2.h5
/content/drive/MyDrive/HLT/clean_dataset_training/model_output/biLSTM/64_0.2_nadam_0.0/model_3.h5
/content/drive/MyDrive/HLT/clean_dataset_training/model_output/biLSTM/64_0.2_nadam_0.0/model_4.h5


11 out of the last 11 calls to <function Model.make_predict_function.<locals>.predict_function at 0x7fb823a914d0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
11 out of the last 11 calls to <function Model.make_predict_function.<locals>.predict_function at 0x7fb82367c200> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing 

[0, 0, 0, 0, 0, 0, 0, 0]

In [113]:
evaluate_with_ALBERTo(dataset_path, model_path_list_ALBERTo)

evaluate_with_ALBERTo
Reading english - 1grams ...
Reading english - 2grams ...
Reading english - 1grams ...


AttributeError: ignored